In [ ]:
#Look for best parameters
import struct
import imghdr


#Function needed to obtain width and height of image
def get_image_size(fname):
    '''Determine the image type of fhandle and return its size.
    from draco'''
    with open(fname, 'rb') as fhandle:
        head = fhandle.read(24)
        if len(head) != 24:
            return
        if imghdr.what(fname) == 'png':
            check = struct.unpack('>i', head[4:8])[0]
            if check != 0x0d0a1a0a:
                return
            width, height = struct.unpack('>ii', head[16:24])
        elif imghdr.what(fname) == 'gif':
            width, height = struct.unpack('<HH', head[6:10])
        elif imghdr.what(fname) == 'jpeg':
            try:
                fhandle.seek(0) # Read 0xff next
                size = 2
                ftype = 0
                while not 0xc0 <= ftype <= 0xcf:
                    fhandle.seek(size, 1)
                    byte = fhandle.read(1)
                    while ord(byte) == 0xff:
                        byte = fhandle.read(1)
                    ftype = ord(byte)
                    size = struct.unpack('>H', fhandle.read(2))[0] - 2
                # We are at a SOFn block
                fhandle.seek(1, 1)  # Skip `precision' byte.
                height, width = struct.unpack('>HH', fhandle.read(4))
            except Exception: #IGNORE:W0703
                return
        else:
            return
        return width, height
    

In [ ]:
# import the necessary packages
from collections import deque
from imutils.video import VideoStream
import numpy as np
import argparse
import cv2
import imutils
from skimage.morphology import erosion, dilation, opening, closing
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
from PIL import Image
from IPython.display import Image as IPImage, display



def compute_center(image, h_threshold, s_threshold, v_threshold, k_erode, k_dilate, iterat_erode, iterat_dilate, ratio_limit, white_threshold, white_pixel_ratio_threshold):
    frame = cv2.imread(image)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create masks for each channel
    h_mask = hsv[:, :, 0] < h_threshold
    s_mask = hsv[:, :, 1] < s_threshold
    v_mask = hsv[:, :, 2] < v_threshold

    combined_mask = v_mask & s_mask & h_mask

    mask_inv = np.ones_like(combined_mask, dtype=np.uint8) * 255
    mask_inv[combined_mask] = 0
    mask = cv2.bitwise_not(mask_inv)

    #Erosion and dilation
    kernel_erode = np.ones((k_erode, k_erode), np.uint8)
    kernel_dilate = np.ones((k_dilate, k_dilate), np.uint8)

    mask1 = cv2.erode(mask, kernel_erode, iterat_erode) 
    mask2 = cv2.dilate(mask1, kernel_dilate, iterat_dilate) 

    # Find contours
    contours, _ = cv2.findContours(mask2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #Make groups
    distances = 999* np.ones((len(contours), len(contours)))

    if len(contours) == 0:
        return 999, 999, 999

    for i in range(len(contours)):
        for j in range(i + 1, len(contours)):
            # Find the centroid of each contour
            M1 = cv2.moments(contours[i])
            centroid1 = (int(M1['m10'] / M1['m00']), int(M1['m01'] / M1['m00']))

            M2 = cv2.moments(contours[j])
            centroid2 = (int(M2['m10'] / M2['m00']), int(M2['m01'] / M2['m00']))

            # Calculate Euclidean distance between centroids
            distance = np.sqrt((centroid1[0] - centroid2[0])**2 + (centroid1[1] - centroid2[1])**2)
            distances[i][j] = distance
            distances[j][i] = distance

    groups = []

    min_distances_indices = np.min(distances, axis=1)
    sourted_indices = np.argsort(min_distances_indices)


    for i in sourted_indices:
        if not (any(i in sublist for sublist in groups)): #checks if that contour hasn't appear yet and calculate its own group
            group = []
            sorted_indices = np.argsort(distances[i])

            max_distance = distances[i, sorted_indices[1]] #Used to compare. The biggest one of the two closest

            group.extend(sorted_indices[:2].tolist())

            for other_index in sorted_indices[2:]:
                for current_index in group:
                    if (distances[current_index, other_index] < ratio_limit * max_distance) and (other_index not in group):
                        group.append(other_index)
            
            groups.append(group)

    #Compute center
    old_best_ratio = 0

    for group in groups:
        group_contours = [contours[i] for i in group]
        group_contour = np.concatenate(group_contours)

        # Calculate minimum enclosing circle
        (x, y), radius = cv2.minEnclosingCircle(group_contour)
        center = (int(x), int(y))
        radius = int(radius)

        # Create a mask for whites inside the area encompassed by the circle
        mask_circle = np.zeros_like(frame, dtype=np.uint8)
        cv2.circle(mask_circle, center, radius, (255, 255, 255), thickness=-1)
        mask_binary = cv2.cvtColor(mask_circle, cv2.COLOR_BGR2GRAY)
        white_pixel_ratio = np.sum(frame[:, :, 2][mask_binary == 255] > white_threshold) / np.sum(mask_binary == 255)

        # Draw the circle only if the white pixel ratio is above the threshold
        if white_pixel_ratio > white_pixel_ratio_threshold and white_pixel_ratio > old_best_ratio:
            old_best_ratio = white_pixel_ratio
            best_radius = radius
            best_center = center
    
    if old_best_ratio == 0:
        return 999, 999, 999
    else:
        best_x, best_y = best_center

    return best_x, best_y, best_radius

In [ ]:
#LOAD FILES OF TEXT


In [ ]:

#SET RANGE OF PARAMETERS WITH WHICH WE WILL PLAY
#lims [min, max, steps]

lim_h_threshold = [0, 250, 2] 
lim_s_threshold = [0, 250, 2]
lim_v_threshold = [0, 250, 2]

lim_k_erode = [0, 10, 1] 
lim_k_dilate = [0, 25, 1] 

lim_iterat_erode = [0, 5, 1] 
lim_iterat_dilate = [0, 5, 1] 

lim_ratio_limit = [1.2, 3, 0.1] 

lim_white_threshold = [150, 250, 2] 
lim_white_pixel_ratio_threshold = [0, 1, 0.1]


combinations = [] #each element will be a list with the parameters of the combination #index of the element. total length = number of combinations
puntuations = [] #each element will be a list of length the number of images in the datalist with 1 when correct and 0 when incorrect. total length = number of combinations x number of images


#Accuracy calculation
for h_threshold in range(lim_h_threshold[0], lim_h_threshold[1], lim_h_threshold[2]):
    for s_threshold in range(lim_s_threshold[0], lim_s_threshold[1], lim_s_threshold[2]):
        for v_threshold in range(lim_v_threshold[0], lim_v_threshold[1], lim_v_threshold[2]):
            for k_erode in range(lim_k_erode[0], lim_k_erode[1], lim_k_erode[2]):
                for k_dilate in range(lim_k_dilate[0], lim_k_dilate[1], lim_k_dilate[2]):
                    for iterat_erode in range(lim_iterat_erode[0], lim_iterat_erode[1], lim_iterat_erode[2]):
                        for iterat_dilate in range(lim_iterat_dilate[0], lim_iterat_dilate[1], lim_iterat_dilate[2]):
                            for ratio_limit in range(lim_ratio_limit[0], lim_ratio_limit[1], lim_ratio_limit[2]):
                                for white_threshold in range(lim_white_threshold[0], lim_white_threshold[1], lim_white_threshold[2]):
                                    for white_pixel_ratio_threshold in range(lim_white_pixel_ratio_threshold[0], lim_white_pixel_ratio_threshold[1], lim_white_pixel_ratio_threshold[2]):

                                        combinations.append([h_threshold, s_threshold, v_threshold, k_erode, k_dilate, iterat_erode, iterat_dilate, ratio_limit, white_threshold, white_pixel_ratio_threshold])
                                        puntuation_of_comb = []

                                        for index in enumerate(dataset):
                                            img = images[index]
                                            target = labels[index]

                                            [x_real, y_real, w_box, h_box] = target

                                            x_calc, y_calc, r_calc = compute_center(img, h_threshold, s_threshold, v_threshold, k_erode, k_dilate, iterat_erode, iterat_dilate, ratio_limit, white_threshold, white_pixel_ratio_threshold)

                                            if correct:
                                                puntuation_of_comb.append(1)
                                            else:
                                                puntuation_of_comb.append(0)
                                        
                                        puntuations.append(puntuation_of_comb)

In [9]:
a = [2,3,4,3,2,2,2,3,4]
p = []
pp = []
import numpy as np
for j in range(0,3):
    for i in a:
        if i == 2:
            p.append(1)
        else:
            p.append(0)
    pp.append(p)
print(np.sum(pp, axis=1))

[12 12 12]
